## On change la loss function --> (multinomial) CrossEntropy loss

## output = multinomial sample

## Passage à une architecture convolutionnelle

Ajouts/modifs :
  * simplification de l'architecture (seule l'image blanchie est en entrée)
  * dans le roll : supression des régions hors image (white noise) 
  
## modif : poids croissant des entrées selon l'eccentricité

## modif : suppression des couches profondes + deconvolution (analyse par niveau de profondeur)


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time


## Torch libraries

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
import torch.utils.data as data

## Parameters

In [3]:
N_theta, N_azimuth, N_eccentricity, N_phase, N_X, N_Y, rho = 6, 24, 16, 2, 256, 256, 1.21 #1.41 #1.25 #
#N_theta, N_azimuth, N_eccentricity, N_phase, N_X, N_Y, rho = 6, 12, 8, 2, 256, 256, 1.41
verbose = 1
OFFSET_STD = 15
OFFSET_MAX = 30

In [4]:
#size used for downsizing images
size=256

test_sample_ratio=5/100
#folder names
imdir_org,fixdir_org='ALLSTIMULI/ALLSTIMULI','ALLFIXATIONMAPS/ALLFIXATIONMAPS'
#imdir_new,fixdir_new='GLOBAL_IMAGES','GLOBAL_FIXATIONMAPS'
#imdir_test,fixdir_test='GLOBAL_IMAGES_TEST','GLOBAL_FIXATIONMAPS_TEST'
#creating empty folders:

imdir, fixdir = 'GLOBAL_IMAGES_ALL_PLAIN','GLOBAL_FIXATIONMAPS_ALL_PLAIN'
imdir_white, fixdir_white = 'GLOBAL_IMAGES_ALL_WHITE','GLOBAL_FIXATIONMAPS_ALL_WHITE'



### Tests

In [5]:
i = 0

'''image_ref_dir = imdir_org
image_ref_names=os.listdir(image_ref_dir)
image_ref=np.load(os.path.join(image_ref_dir,image_ref_names[i]))

fix_ref_dir = fixdir_org
fix_ref_names=os.listdir(fix_ref_dir)
fix_ref=np.load(os.path.join(fix_ref_dir,fix_ref_names[i]))'''

image_dir = imdir #imdir_new
image_names=os.listdir(image_dir)
image=np.load(os.path.join(image_dir,image_names[i]))

image_dir_white = imdir_white #imdir_new
image_white_names=os.listdir(image_dir_white)
image_white=np.load(os.path.join(image_dir_white,image_names[i]))

fix_dir = fixdir #fixdir_new
fix_names=os.listdir(fix_dir)
fix=np.load(os.path.join(fix_dir,fix_names[i]))


FileNotFoundError: [Errno 2] No such file or directory: 'GLOBAL_IMAGES_ALL_PLAIN'

In [ ]:
plt.figure(figsize = (10,3))
'''plt.subplot(141)
plt.imshow(image_ref)
plt.subplot(142)
_ = plt.plot(fix_ref)'''
plt.subplot(141)
plt.imshow(image)
plt.subplot(142)
_ = plt.plot(image)
plt.subplot(143)
plt.imshow(fix)
plt.subplot(144)
_ = plt.plot(fix/sum(fix.flatten()))

In [ ]:
plt.imshow(fix)
m = fix/sum(fix.flatten())
m = m.flatten()

'''for i in range(20):
    print(np.where(np.random.multinomial(1, m) == 1)[0][0])'''

m_mult = np.random.multinomial(2000, m)
plt.figure()
plt.imshow(m_mult.reshape(256, 256))

m_mult = np.random.multinomial(1, m).reshape(256, 256)
print(np.where(m_mult == 1))
coord = np.where(m_mult == 1)
plt.plot(coord[1], coord[0], 'r+')
print(coord[1][0])
    

In [ ]:
plt.figure(figsize = (10,3))
'''plt.subplot(141)
plt.imshow(image_ref)
plt.subplot(142)
_ = plt.plot(fix_ref)'''
plt.subplot(141)
plt.imshow(image_white)
plt.subplot(142)
_ = plt.plot(image_white)
plt.subplot(143)
plt.imshow(fix)
plt.subplot(144)
_ = plt.plot(fix)

## Vision stuff

In [ ]:
from LogGabor import LogGabor

## Pierre's stuff

### Encoding : N_theta x N_azimuth x N_eccentricity x N_phase  2D filters (to be applied on N_X x N_Y pixels)

#### Préparer l'apprentissage et les fonctions nécessaires au fonctionnement du script

In [ ]:
def vectorization(N_theta=N_theta, N_azimuth=N_azimuth, N_eccentricity=N_eccentricity, N_phase=N_phase, \
                  N_X=N_X, N_Y=N_Y, rho=rho, ecc_max=1, B_sf=.4, B_theta=np.pi/N_theta/2):
    retina = np.zeros((N_theta, N_azimuth, N_eccentricity, N_phase, N_X*N_Y))
    parameterfile = 'https://raw.githubusercontent.com/bicv/LogGabor/master/default_param.py'
    lg = LogGabor(parameterfile)
    lg.set_size((N_X, N_Y))
    # params = {'sf_0': .1, 'B_sf': lg.pe.B_sf,
    #           'theta': np.pi * 5 / 7., 'B_theta': lg.pe.B_theta}
    # phase = np.pi/4
    # edge = lg.normalize(lg.invert(lg.loggabor(
    #     N_X/3, 3*N_Y/4, **params)*np.exp(-1j*phase)))

    for i_theta in range(N_theta):
        for i_azimuth in range(N_azimuth):
            for i_eccentricity in range(N_eccentricity):
                ecc = ecc_max * (1/rho)**(N_eccentricity - i_eccentricity)
                r = np.sqrt(N_X**2+N_Y**2) / 2 * ecc  # radius
                sf_0 = 0.5 * 0.03 / ecc
                x = N_X/2 + r * \
                    np.cos((i_azimuth+(i_eccentricity % 2)*.5)*np.pi*2 / N_azimuth)
                y = N_Y/2 + r * \
                    np.sin((i_azimuth+(i_eccentricity % 2)*.5)*np.pi*2 / N_azimuth)
                for i_phase in range(N_phase):
                    params = {'sf_0': sf_0, 'B_sf': B_sf,
                              'theta': i_theta*np.pi/N_theta, 'B_theta': B_theta}
                    phase = i_phase * np.pi/2
                    # print(r, x, y, phase, params)

                    retina[i_theta, i_azimuth, i_eccentricity, i_phase, :] = lg.normalize(
                        lg.invert(lg.loggabor(x, y, **params)*np.exp(-1j*phase))).ravel() * 2 * np.pi * ecc


    return retina

#### Full encoding

In [ ]:
#del retina

In [ ]:
FIC_NAME = 'retina_256_24_ecc_16_expand.npy'
if not os.path.exists(FIC_NAME):
    retina = vectorization(N_theta, N_azimuth, N_eccentricity, N_phase, N_X, N_Y, rho) #, ecc_max=1)
    np.save(FIC_NAME, retina)
else:
    retina = np.load(FIC_NAME)


In [ ]:
print(retina.shape)

In [ ]:
for i in range(16):
    plt.hist(abs(retina[:,:,15-i,:,::10000].flatten()), 100)
    plt.legend()
    plt.ylim((0,500))

In [ ]:
retina_vector = retina.reshape((N_theta*N_azimuth*N_eccentricity*N_phase, N_X*N_Y))
print(retina_vector.shape)

In [ ]:
FIC_NAME = 'retina_inverse_256_24_ecc_16_expand.npy'
if not os.path.exists(FIC_NAME):
    retina_inverse = np.linalg.pinv(retina_vector)
    np.save(FIC_NAME, retina_inverse)
else:
    retina_inverse = np.load(FIC_NAME)
print(retina_inverse.shape)

#### Orientation invariant power encoding (colliculus??)

In [ ]:
colliculus = (retina**2).sum(axis=(0, 3))
colliculus = colliculus**.5
colliculus /= colliculus.sum(axis=-1)[:, :, None]
print(colliculus.shape)

In [ ]:
colliculus_vector = colliculus.reshape((N_azimuth*N_eccentricity, N_X*N_Y))
print(colliculus_vector.shape)

In [ ]:
colliculus_inverse = np.linalg.pinv(colliculus_vector)
print(colliculus_inverse.shape)

In [ ]:
energy = colliculus ** 2
energy /= energy.sum(axis=-1)[:, :, None]
energy_vector = energy.reshape((N_azimuth*N_eccentricity, N_X*N_Y))
energy_plus = np.linalg.pinv(energy_vector)
FIG_WIDTH = 5 

fig, ax = plt.subplots(figsize=(FIG_WIDTH, FIG_WIDTH))
for i_orient in range(N_azimuth):
    for i_scale in range(N_eccentricity):
        env = np.sqrt(energy[i_orient, i_scale, :]**2.5).reshape((N_X, N_Y))
        ax.contour(energy[i_orient, i_scale, :].reshape((N_X, N_Y)), levels=[env.max()/2], lw=1,
                  colors=[plt.cm.rainbow(i_scale * 1.5/N_azimuth)])
fig.suptitle('Tiling of visual space using energy')
ax.set_xlabel(r'$Y$')
ax.set_ylabel(r'$X$')
ax.axis('equal')
plt.tight_layout()

## Images dataset

In [ ]:
def minmax(value, border):
    value = max(value, -border)
    value = min(value, border)
    return int(value)

In [ ]:
#im = SLIP.Image(pe='https://raw.githubusercontent.com/bicv/LogGabor/master/default_param.py')

class Transform(object):
    """Rescale the image through LogGabors

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, test = False, tensor = False):
        #assert isinstance(output_size, (int, tuple))
        #self.output_size = output_size
        self.test = test
        self.tensor = tensor

    def __call__(self, sample):
        image, image_white, fixmap = sample['image'], sample['image_white'], sample['fixation']
        (nb_l, nb_col) = image.shape
        
        #i_offset = minmax(np.random.randn() * OFFSET_STD, OFFSET_MAX)
        #j_offset = minmax(np.random.randn() * OFFSET_STD, OFFSET_MAX)
        
        '''if np.random.rand() > 0.5:
            image_white = np.fliplr(image_white)
            image = np.fliplr(image)
            fixmap = np.fliplr(fixmap)'''
        
        if True : #not self.test:
            m = fixmap/sum(fixmap.flatten())
            m = m.flatten()
            m_mult = np.random.multinomial(1, m).reshape(256, 256)
            coord = np.where(m_mult == 1)
            i_offset = 127 - coord[0][0]
            j_offset = 127 - coord[1][0]
        else:
            i_offset = 0
            j_offset = 0
        
        '''i_offset = 0
        j_offset = -50'''
        
        image_white = np.roll(image_white, i_offset, axis=0)
        image_white = np.roll(image_white, j_offset, axis=1)  
        mean_white = np.mean(image_white.flatten())
        std_white = np.std(image_white.flatten())
        
        image = np.roll(image, i_offset, axis=0)
        image = np.roll(image, j_offset, axis=1)
        mean_im = np.mean(image.flatten())
        std_im = np.std(image.flatten())
        
        fixmap = np.roll(fixmap, i_offset, axis=0)
        fixmap = np.roll(fixmap, j_offset, axis=1)    
        
        if i_offset > 0 :
            image_white[:i_offset, :] = mean_white + std_white * np.random.randn(i_offset, nb_col) #127
            image[:i_offset, :] = mean_im + std_im * np.random.randn(i_offset, nb_col) #127
            fixmap[:i_offset, :] = 0
        elif i_offset < 0:
            image_white[i_offset:, :] = mean_white + std_white * np.random.randn(-i_offset, nb_col) #127
            image[i_offset:, :] = mean_im + std_im * np.random.randn(-i_offset, nb_col) #127
            fixmap[i_offset:, :] = 0     
        
        if j_offset > 0 :
            image_white[:,:j_offset] = mean_white + std_white * np.random.randn(nb_l, j_offset) #127
            image[:,:j_offset] = mean_im + std_im * np.random.randn(nb_l, j_offset) #127
            fixmap[:,:j_offset] = 0
        elif j_offset < 0:
            image_white[:,j_offset:] = mean_white + std_white * np.random.randn(nb_l, -j_offset) 
            image[:,j_offset:] = mean_im + std_im * np.random.randn(nb_l, -j_offset) 
            fixmap[:,j_offset:] = 0
        
        image_white = (image_white - mean_white) / std_white
        image = (image - mean_im) / std_im
        
        '''plt.figure()    
        plt.imshow(image_white, cmap = 'gray')
        plt.figure()    
        plt.imshow(fixmap, cmap = 'gray')'''
        
        image_retina = retina_vector @ np.ravel(image_white)
        image_retina /= np.std(image_retina)
        
        image_colliculus = colliculus_vector @ np.ravel(image)
        #image_colliculus -= np.mean(image_colliculus)
        image_colliculus /= np.std(image_colliculus)
        
        fixmap_colliculus = colliculus_vector @ np.ravel(fixmap)
        fixmap_colliculus = fixmap_colliculus/np.sum(fixmap_colliculus)
        
        if not self.test:
            m_coll = fixmap_colliculus/sum(fixmap_colliculus)
            m_coll_mult = np.random.multinomial(25, m_coll)
            #m_coll_mult[np.where(m_coll_mult > 1)] = 1
            m_coll_mult = np.array(m_coll_mult)/np.sum(m_coll_mult)
        
        if self.tensor:
            image_retina = image_retina.reshape(N_theta, N_azimuth, N_eccentricity, N_phase)
            slice1 = image_retina[N_theta - 1,:,:,:].reshape(1,N_azimuth,N_eccentricity,N_phase)
            slice2 = image_retina[0,:,:,:].reshape(1,N_azimuth,N_eccentricity,N_phase)
            image_retina = np.concatenate ((slice1, image_retina, slice2), axis = 0)
            image_retina = np.transpose(image_retina,(3,0,1,2))
            image_colliculus = image_colliculus.reshape(1,N_azimuth, N_eccentricity)
            '''if not self.test:
                m_coll_mult = m_coll_mult.reshape(1, N_azimuth, N_eccentricity)
            else:
                fixmap_colliculus = fixmap_colliculus.reshape(N_azimuth, N_eccentricity)'''
                
        if self.test:
            return {'image': image_colliculus, 'image_white': image_retina, 'fixation': fixmap_colliculus}
        else:
            return {'image': image_colliculus, 'image_white': image_retina, 'fixation': m_coll_mult}   
        
        #r 

In [ ]:
## test

In [ ]:
class ImageDataset(data.Dataset):
    """image dataset."""

    def __init__(self, imdir, imdir_white, fixdir, transform=None, index = None):
        """
        Args:
            imdir (string): Path to the image folder
            fixdir (string): Path to the fixation maps folder
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.imdir = imdir
        self.fixdir = fixdir
        self.imdir_white = imdir_white
        if index is None :
            print('OK')
            self.image_names=os.listdir(imdir)
            self.fix_names=os.listdir(fixdir)
        else:
            self.image_names=np.array(os.listdir(imdir))[index]
            self.fix_names=np.array(os.listdir(fixdir))[index]
        self.transform = transform # we do not use transforms
        #self.data_loader=data.DataLoader(self,batch_size=batch_size) : did not work
        
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image=np.load(os.path.join(self.imdir,self.image_names[idx]))
        image_white=np.load(os.path.join(self.imdir_white,self.image_names[idx]))
        fix_map=np.load(os.path.join(self.fixdir,self.fix_names[idx]))/255 # to transform between 0 and 1 (for the BCELoss to work)

        sample = {'image': image, 'image_white': image_white, 'fixation': fix_map}

        if self.transform:
            sample = self.transform(sample)
        
        return sample


### Tests

In [ ]:
trans = Transform(tensor = False)
dataset = ImageDataset(image_dir,image_dir_white,fix_dir)

In [ ]:
i = 0
sample = dataset[i] #['image'], dataset[i]['fixation'] 

In [ ]:
plt.figure(figsize = (10,3))
plt.subplot(121)
plt.imshow(sample['image'], cmap = 'gray')
plt.subplot(122)
_ = plt.plot(sample['image'])

In [ ]:
im = sample['image']
im2 = np.pad(im, ((10, 10), (10, 10)), 'median')
plt.imshow(im2)

In [ ]:
image_off = sample['image']
i_offset = minmax(np.random.randn() * OFFSET_STD, OFFSET_MAX)
j_offset = minmax(np.random.randn() * OFFSET_STD, OFFSET_MAX)
print(i_offset, j_offset)
print(OFFSET_STD)
image_off = np.roll(image_off, i_offset, axis=0)
image_off = np.roll(image_off, j_offset, axis=1)    
if i_offset > 0 :
    image_off[:i_offset, :] = 0
elif i_offset < 0:
    image_off[i_offset:, :] = 0

if j_offset > 0 :
    image_off[:,:j_offset] = 0
elif j_offset < 0:
    image_off[:,j_offset:] = 0

plt.figure(figsize = (10,3))
plt.subplot(121)
plt.imshow(sample['image'], cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')
plt.subplot(122)
plt.imshow(image_off, cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

In [ ]:
plt.figure(figsize = (10,3))
plt.subplot(121)
plt.imshow(sample['image_white'], cmap = 'gray')
plt.subplot(122)
_ = plt.plot(sample['image_white'])

In [ ]:
'''train = True
batch_size = 10
data_loader = torch.utils.data.DataLoader(dataset, train=train, download=True, batch_size=batch_size, shuffle=True)'''
dataloader = data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=4)

In [ ]:
batch = next(iter(dataloader))

In [ ]:
batch['image'].shape

In [ ]:
plt.figure(figsize = (10,3))
plt.subplot(121)
plt.imshow(batch['image'][0,:,:], cmap = 'gray')
plt.subplot(122)
_ = plt.plot(batch['image'][0,:,:])

### Encoding test

In [ ]:
trans = Transform(tensor = True)
sample_retina = trans(sample) #retina_vector @ np.ravel(sample['image'])
sample_cut = sample_retina['image_white'][:,1:N_theta+1,:,:]
print(sample_cut.shape)
sample_cut = np.transpose(sample_cut, (1,2,3,0))
print(sample_cut.shape)
plt.figure()
plt.plot(sample_cut.reshape(N_azimuth * N_eccentricity * N_theta * N_phase))

In [ ]:
np.mean(sample_cut.reshape(N_azimuth * N_eccentricity * N_theta * N_phase))

In [ ]:
trans = Transform()
sample_retina = trans(sample) #retina_vector @ np.ravel(sample['image'])
plt.plot(sample_retina['image'].reshape(N_azimuth * N_eccentricity))

In [ ]:
#data_colliculus = colliculus_vector @ np.ravel(sample['fixation'])
plt.plot(sample_retina['fixation'].reshape(N_azimuth * N_eccentricity))

In [ ]:
### Polar plot

In [ ]:
trans = Transform(test = False)
sample_retina = trans(sample)
delta = 1/N_azimuth
log_r, theta = np.meshgrid(np.linspace(0, 1, N_eccentricity + 1), np.linspace(-np.pi*(.5 + delta), np.pi*(1.5 - delta), N_azimuth + 1))
f = plt.figure(figsize = (15,5))

plt.subplot(131)
data_retina = sample_retina['image_white'].reshape(N_azimuth * N_eccentricity * N_theta * N_phase)
#data_retina = sample_cut.reshape(N_azimuth * N_eccentricity * N_theta * N_phase)
image_white = retina_inverse @ data_retina
#im = colliculus_inverse @ data_retina
plt.imshow(image_white.reshape(N_X, N_Y), cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

ax = f.add_subplot(132, projection='polar')
vec = sample_retina['fixation'].reshape(N_azimuth * N_eccentricity)
ax.pcolor(theta, log_r, vec.reshape((N_azimuth, N_eccentricity)))
plt.plot(0,0, 'r+')

ax = f.add_subplot(133)
plt.imshow((colliculus_inverse @ vec).reshape(N_X, N_Y) )#sample['fixation'])
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

In [ ]:
colliculus_inverse @ vec

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(121)
plt.imshow(sample['image_white'], cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

plt.subplot(122)
data_retina = sample_retina['image_white'].reshape(N_azimuth * N_eccentricity * N_theta * N_phase)
image_white = retina_inverse @ data_retina
#im = colliculus_inverse @ data_retina
plt.imshow(image_white.reshape(N_X, N_Y), cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(121)
plt.imshow(sample['image'], cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

plt.subplot(122)
data_retina = sample_retina['image'].reshape(N_azimuth * N_eccentricity)
image = colliculus_inverse @ data_retina
#im = colliculus_inverse @ data_retina
plt.imshow(image.reshape(N_X, N_Y), cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

In [ ]:
plt.figure(figsize = (10,5))
plt.subplot(121)
plt.imshow(sample['image'], cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')

plt.subplot(122)
plt.imshow(10 * image_white.reshape(N_X, N_Y)+image.reshape(N_X, N_Y), cmap = 'gray')
plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')


In [ ]:
if False:
    transform = Transform()
    batch_size = 100
    dataset = ImageDataset(image_dir, image_dir_white, fix_dir, transform = transform)
    dataloader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    batch = next(iter(dataloader))

In [ ]:
if False :
    fixmap_avg = sum(batch['fixation'])/batch_size

    f = plt.figure(figsize = (15, 5))

    i = 3

    plt.subplot(131)
    #for i in range(10):
    plt.plot(batch['fixation'][i,:], 'g')
    plt.plot(batch['fixation'][i,:] - fixmap_avg)
    plt.plot(fixmap_avg,'r')

    ax = f.add_subplot(132, projection='polar')
    vec = batch['fixation'][i,:] 
    ax.pcolor(theta, log_r, vec.reshape((N_azimuth, N_eccentricity)))
    plt.plot(0,0, 'r+')

    ax = f.add_subplot(133, projection='polar')
    vec = batch['fixation'][i,:] - fixmap_avg
    ax.pcolor(theta, log_r, vec.reshape((N_azimuth, N_eccentricity)))
    plt.plot(0,0, 'r+')



#### Input vectors encoding

In [ ]:
'''def image_fullfield(image):
    image_retina = retina_vector @ np.ravel(image)
    return image_retina'''

#### Output vectors (Accuracy/Saliency map) encoding

In [ ]:
'''def fixmap_fullfield(fixmap):   
    fixmap_colliculus = colliculus_vector @ np.ravel(fixmap)
    return fixmap_colliculus'''

In [ ]:
'''def couples(image, fixmap): #, device):
    #data = data.to(device)
    v = image_fullfield(data)
    a = fixmap_fullfield(fixmap)
    return (v, a)'''

#### Hyperparameters

In [ ]:
minibatch_size = 25  # quantity of examples that'll be processed
lr = 1e-4 #0.05

verbose = 1
train = True

In [ ]:
torch.cuda.is_available()

In [ ]:
do_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 4, 'pin_memory': True} if do_cuda else {}
device = torch.cuda.device("cuda" if do_cuda else "cpu")

In [ ]:
transform = Transform(tensor = True)
transform_test = Transform(tensor = True, test = True)

image_dir = imdir #'GLOBAL_IMAGES_ALL'
fix_dir = fixdir #'GLOBAL_FIXATIONMAPS_ALL'

'''image_names=os.listdir(image_dir)
print (len(image_names))
fix_names=os.listdir(fix_dir)
print (len(fix_names))
for i in range(len(image_names)):
    if image_names[i][:7] != fix_names[i][:7]:
        print(image_names[i], fix_names[i])'''

n = len(os.listdir(image_dir))
index = np.arange(n)
np.random.shuffle(index)
print(index)
index_train = index[:800]
index_test = index[800:]

train_dataset = ImageDataset(image_dir, image_dir_white, fix_dir, transform = transform, index = index_train)
train_loader = data.DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True, num_workers=4)

test_dataset = ImageDataset(image_dir, image_dir_white, fix_dir, transform = transform_test, index = index_test)
test_loader = data.DataLoader(test_dataset, batch_size = len(test_dataset), shuffle=True, num_workers=4)

In [ ]:
len(train_dataset)

In [ ]:
'''for batch_idx, batch in enumerate(train_loader):
    print (batch_idx)'''

In [ ]:
N_azimuth, N_eccentricity, N_theta

#### Network

In [ ]:
BIAS_CONV = True
BIAS_DECONV = True #True

class Net(torch.nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        ## White
        self.conv1 = nn.Conv3d(2, 16, 3, bias = BIAS_CONV, stride=1, padding=1)
        self.conv2 = nn.Conv3d(16, 64, 3, bias = BIAS_CONV, stride=1, padding=1)
        self.conv3 = nn.Conv3d(64, 256, 3, bias = BIAS_CONV, stride=1, padding=1)
        self.pool = nn.MaxPool3d(2, stride=2)
        self.conv_hidden1 = nn.Conv2d(256, 20, 1, bias = BIAS_CONV, stride=1, padding=0)
        self.conv_hidden2 = nn.Conv2d(20, 2, 1, bias = BIAS_CONV, stride=1, padding=0)
        self.deconv_hidden2 = nn.Conv2d(2, 100, 1, bias = BIAS_DECONV, stride=1, padding=0)
        self.deconv_hidden1 = nn.Conv2d(100, N_azimuth / 3 * N_eccentricity / 2, 1, bias = BIAS_DECONV, stride=1, padding=0)
        # taille 256 *  3 (az) * 2 (ecc) * 1 (thet)
        
        
        #self.dropout = nn.Dropout(p = 0.5) 
        
    def forward(self, image):
        
        # in
        data = F.relu(self.pool(self.conv1(image)))
        data = F.relu(self.pool(self.conv2(data)))
        data = F.relu(self.pool(self.conv3(data)))
        data = data.view(-1, 256, 3, 2)
        
        data = F.relu(self.conv_hidden1(data)) 
        data = F.dropout(data, p = .5) 
        data = self.conv_hidden2(data)
        data = F.relu(self.deconv_hidden2(data))
        data = self.deconv_hidden1(data)
        
        # out
        for i in range(3): # Azimuth
            for j in range(2): # Eccentricity
                data_cell = data[:,:,i,j].view(-1, N_azimuth / 3, N_eccentricity / 2) 
                if j == 0 :
                    data_row = data_cell
                else:
                    data_row = torch.cat((data_row, data_cell), 2)
            if i == 0 :
                data_full = data_row
            else:
                data_full = torch.cat((data_full, data_row), 1)
        data = data_full.view(-1, N_azimuth * N_eccentricity) 
        return data
    
    '''def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features'''

In [ ]:
net = Net()

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
#loss_func = torch.nn.CrossEntropyLoss() #BCEWithLogitsLoss()

def loss_func(pred, soft_targets):
    # cross entropy
    logsoftmax = nn.LogSoftmax()
    return torch.mean(torch.sum(- soft_targets * logsoftmax(pred), 1))

In [ ]:
#loss_func = torch.nn.BCEWithLogitsLoss()
def KL_loss(pred, soft_targets):
    # cross entropy
    logsoftmax = nn.LogSoftmax()
    log = torch.log
    return torch.mean(torch.sum(- soft_targets * (logsoftmax(pred) - log(soft_targets)), 1))

In [ ]:
#loss_func = torch.nn.BCEWithLogitsLoss()
def entropy(soft_targets):
    # cross entropy
    log = torch.log
    return -torch.mean(torch.sum(soft_targets *  log(soft_targets), 1))

In [ ]:
def train(net, minibatch_size, \
          optimizer=optimizer, \
          vsize = N_theta * N_azimuth * N_eccentricity * N_phase,\
          asize = N_azimuth * N_eccentricity, \
          verbose=1):
    
    t_start = time.time()
    
    if verbose: print('Starting training...')
    
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()
       
        log_image_batch_white = batch['image_white'].float()
        log_fixmap_batch = batch['fixation'].float()
        
        prediction = net(log_image_batch_white)
        loss = loss_func(prediction, log_fixmap_batch) 

        loss.backward()
        optimizer.step()

        if verbose and batch_idx % 10 == 0:
            print('[{}/{}] Loss: {} Time: {:.2f} mn'.format(
                batch_idx * minibatch_size, len(train_loader.dataset),
                loss.data.numpy(), (time.time()-t_start)/60))
    return net


In [ ]:
def test(net, minibatch_size, optimizer=optimizer,
         vsize=N_theta*N_azimuth*N_eccentricity*N_phase,
         asize=N_azimuth*N_eccentricity):
    #for batch_idx, (data, label) in enumerate(test_loader):
    batch = next(iter(test_loader))
    batch_size = len(batch)
    
    '''log_image_batch = np.zeros((batch_size, 1, vsize))
    log_fixmap_batch = np.zeros((batch_size, 1, asize))
    
    for idx in range(batch_size):
        log_image_batch[idx, 0, :], log_fixmap_batch[idx, 0, :] = couples(batch['image'][idx, :, :], batch['fixation'][idx, :, :])
        #input_[idx, 0, :], a_data[idx, 0, :] = couples(data[idx, 0, :], i_offset, j_offset)
        #target[idx, :] = a_data[idx, 0, :]'''


    prediction = net(batch['image_white'].float())
    loss = loss_func(prediction, batch['fixation'].float())

    return loss.data.numpy()


In [ ]:
#FIC_NAME = 'results/2019-01-01-Malek-recap-multi-1-no-offset'
FIC_NAME = 'results/2019-01-08-Malek-recap-multi-25-withbias'
EPOCHS = 50

if not os.path.exists(FIC_NAME + '.npy'):
    for epoch in range(EPOCHS) :
        print(epoch)
        train(net, minibatch_size)
        Accuracy = test(net, minibatch_size)
        mes = 'Test set: Final Accuracy: {:.3f}'.format(Accuracy * 1.) 
        print(mes)
        with open(FIC_NAME + '.txt', 'a') as f:
            f.write(mes + '\n')
        torch.save(net, FIC_NAME + '.npy')    
        np.save(FIC_NAME + '-index.npy', index)   
else:
    net = torch.load(FIC_NAME + '.npy')    

In [ ]:
if False:
    np.save('2018-12-23-fixation-ref.npy', vec_ref)
else:
    vec_ref = np.load('2018-12-23-fixation-ref.npy')

In [ ]:
NB_FIG = 20
NB_TURN = 1 #1000
FLAG_AFF = True

delta = 1/N_azimuth
log_r, theta = np.meshgrid(np.linspace(0, 1, N_eccentricity + 1), np.linspace(-np.pi*(.5 + delta), np.pi*(1.5 - delta), N_azimuth + 1))

class Results:
    def __init__(self):
        pass

results = Results()

if not os.path.exists(FIC_NAME+'-results.npy') or FLAG_AFF: 
    index = np.load(FIC_NAME+'-index.npy')
    index_train = index[:800] 
    index_test = index[800:] 
    net = torch.load(FIC_NAME+'.npy', map_location=lambda storage, loc: storage) #torch.load(FIC_NAME+'.npy').to('cpu') 

    mem_loss_base, mem_loss_out, mem_entropy_fix = [], [], []
    mem_KL_loss_base, mem_KL_loss_out = [], []
    mem_KL_diff, mem_relative_gain = [], []
    for __ in range(NB_TURN):
        print(__)
        if False :
            transform_train = Transform(test = True, tensor = True) 
            train_dataset = ImageDataset(image_dir, image_dir_white, fix_dir, transform = transform_train, index = index_train)
            train_loader = data.DataLoader(train_dataset, batch_size = NB_FIG , shuffle=True, num_workers=4)
            batch = next(iter(train_loader))
        else:
            transform_test = Transform(test = True, tensor = True) 
            test_dataset = ImageDataset(image_dir, image_dir_white, fix_dir, transform = transform_test, index = index_test)
            test_loader = data.DataLoader(test_dataset, batch_size = NB_FIG , shuffle=True, num_workers=4)
            batch = next(iter(test_loader))

        vec_ref_torch = Variable(torch.FloatTensor(vec_ref))
        for _ in range(NB_FIG ):
         
            vec = batch['fixation'][_,:]
            in_white = batch['image_white'][_,:].reshape((1,N_phase,N_theta+2,N_azimuth,N_eccentricity)).float()
            out = net(in_white)

            loss_base = loss_func(torch.log(vec_ref_torch).reshape((1, -1)), vec.float()).detach().numpy()
            mem_loss_base += [loss_base]
            loss_out = loss_func(out, vec.float()).detach().numpy()
            mem_loss_out += [loss_out]
            entropy_fix = entropy(vec.reshape(1,-1)).detach().numpy()
            mem_entropy_fix += [entropy_fix]
            KL_loss_base = KL_loss(torch.log(vec_ref_torch).reshape((1, -1)), vec.float()).detach().numpy()
            mem_KL_loss_base += [KL_loss_base]
            KL_loss_out = KL_loss(out, vec.float()).detach().numpy()
            mem_KL_loss_out += [KL_loss_out]
            KL_diff = KL_loss_base - KL_loss_out
            mem_KL_diff +=  [KL_diff]
            relative_gain = 1 - KL_loss_out/KL_loss_base
            mem_relative_gain += [relative_gain]

            if FLAG_AFF:
                
                image = colliculus_inverse @ batch['image'][_,:].reshape(N_azimuth * N_eccentricity)
                
                data_white = batch['image_white'][_,:,1:N_theta+1,:,:]
                data_white = np.transpose(data_white, (1, 2, 3, 0))
                image_white = retina_inverse @ data_white.reshape(N_theta * N_azimuth * N_eccentricity * N_phase)
                
                plt.figure(figsize = (15, 5))

                #plt.subplot(161)
                #plt.plot(batch['image_white'][_,:])
                plt.subplot(131)
                plt.imshow(image_white.reshape(N_X, N_X) * 20 + image.reshape(N_X, N_X) * 0, cmap = 'gray')
                plt.plot(N_X/2 - 0.5, N_Y/2 - 0.5, 'r+')
                plt.title(_)
                plt.subplot(163)
                plt.plot(batch['fixation'][_,:].detach().numpy()[::-1])    
                plt.title('Loss_base : %.2f' % loss_base)
                #col = colliculus_inverse @ batch['fixation'][_,:]
                ax = plt.subplot(264, projection='polar')

                ax.pcolor(theta, log_r, vec.reshape((N_azimuth, N_eccentricity)))
                plt.title('Loss : %.2f' % loss_out)

                plt.subplot(2,6,10)
                map = vec.reshape((N_azimuth, N_eccentricity)).detach().numpy().transpose()
                map = np.flipud(np.fliplr(map))
                plt.imshow(map)
                plt.title('Entropy : %.2f' % entropy_fix)

                #plt.imshow(col.reshape(128, 128))

                plt.subplot(165)
                out_sig = F.softmax(out).detach().numpy()
                plt.plot(out_sig[0,::-1]) #) #_sig)    
                loss = loss_func(out, vec.float()).detach().numpy()
                plt.title("KL_base = %.2f"% KL_loss_base)
                #view = colliculus_inverse @ out_sig
                ax = plt.subplot(266, projection='polar')
                ax.pcolor(theta, log_r, out_sig.reshape((N_azimuth, N_eccentricity)))
                plt.plot(0,0,'r+')
                #plt.imshow(view.reshape(128, 128))
                plt.title("KL_out = %.2f" % KL_loss_out)
                plt.subplot(2,6,12)
                map = out_sig.reshape((N_azimuth, N_eccentricity)).transpose()
                map = np.flipud(np.fliplr(map))
                plt.imshow(map)
                #plt.title("KL_diff = %.2f" % KL_diff)
                plt.title("relative gain = %.2f" % relative_gain)
    
        if not FLAG_AFF :
            results.mem_loss_base = mem_loss_base
            results.mem_loss_out = mem_loss_out
            results.mem_entropy_fix = mem_entropy_fix
            results.mem_KL_loss_base = mem_KL_loss_base
            results.mem_KL_loss_out = mem_KL_loss_out
            results.mem_KL_diff = mem_KL_diff
            results.mem_relative_gain = mem_relative_gain
            np.save(FIC_NAME+'-results.npy', results)
else:
    results = np.load(FIC_NAME+'-results.npy')
